In [1]:
# disabling contracts for speed
import contracts
import yaml
import numpy as np
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
import duckietown_world as dw
from copy import deepcopy
import geometry as g
import pandas as pd

contracts.disable_all()
m = dw.load_map('ethz_amod_lab_k31')


INFO:dt-world:duckietown-world 1.0.30
INFO:zj:zj 2.0.4
INFO:dt-world:contracts 1.8.12 


ha


INFO:dt-world:loading map ethz_amod_lab_k31


In [2]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

class Person(dw.PlacedObject):
    def __init__(self, radius, *args, **kwargs):

        self.radius = radius

        dw.PlacedObject.__init__(self, *args, **kwargs)

        

    def draw_svg(self, drawing, g):

        # drawing is done using the library svgwrite

        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')

        g.add(c)

        # draws x,y axes

        dw.draw_axes(drawing, g)


    def extent_points(self):

        # set of points describing the boundary 

        L = self.radius

        return [(-L, -L), (+L, +L)]

def relative_pose(q0, q1):
    return g.SE2.multiply(g.SE2.inverse(q0), q1)

def interpolate(q0, q1, alpha):
    q1_from_q0 = relative_pose(q0, q1)
    vel = g.SE2.algebra_from_group(q1_from_q0)
    rel = g.SE2.group_from_algebra(vel * alpha)
    q = g.SE2.multiply(q0, rel)
    return q

In [3]:
root = dw.PlacedObject()
import sys
import traceback



realTimestamps = []
with open("trajectory2Pose/autobot29_v3/autobot29_v3.yaml", 'r') as stream:

    try:

        data = yaml.safe_load(stream)

    except yaml.YAMLError as exc:

        print(exc)

    

    timestart = data['begin_time_stamp']
    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))

    y= np.zeros((data_points,))

    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    

    dx = 999.999*np.ones((data_points, ))

    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))

    final_trajectory = []

    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))
        

        phi[:,idx] = np.array([np.arctan2(-R[1,2,idx],R[2,2,idx]), 
                               np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),
                               np.arctan2(-R[0,1,idx], R[0,0,idx])])

        realTimestamps.append(time)
        #print(phi[:,idx])
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    final_array = final_trajectory
    
seqs2 = []

for entry in range(0, len(final_array)):
    x =  (final_array[entry][0][0] )  # -2.2
    y = final_array[entry][0][1] # + 0.8
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)



In [6]:

timestamps = range(len(seqs2)) # [0, 1, 2, ...]
# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
seq_me = dw.SampledSequence(timestamps, transforms)
counter = 0
center_points2 = []
width = 0

for timestamp, pose_object in seq_me:
    hoi = list(get_lane_poses(m, pose_object.as_SE2()))
    
    if len(hoi) == 0:
        # Now has only zero length when in asphalt
        print(' ')
        #print(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
        #center_points2.append(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([0,0],0)))
        
    else:
        #print('outside left: ' + str(hoi[0].lane_pose.distance_from_left) + ' outside right: ' + str(hoi[0].lane_pose.distance_from_right))
        distance_from_left = hoi[0].lane_pose.distance_from_left
        distance_from_right = hoi[0].lane_pose.distance_from_right
        distance_from_center = hoi[0].lane_pose.distance_from_center
        rel_heading = hoi[0].lane_pose.relative_heading
        alongInside = hoi[0].lane_pose.along_inside
        outsideLeft = hoi[0].lane_pose.outside_left
        inside = hoi[0].lane_pose.inside
        lateral = hoi[0].lane_pose.lateral
        lateral_left = hoi[0].lane_pose.lateral_left
        width = lateral_left
        tile = list(hoi[0].tile.children.keys())[0]

        correctDir = hoi[0].lane_pose.correct_direction
        print(timestamp, inside, distance_from_center)
        #print(timestamp, distance_from_center, rel_heading) 
        
        #absTime = float(relTimestamps[counter]) + int(timestart)
        center_points2.append(hoi[0].lane_pose.center_point)
        #print(hoi[0].lane_pose.distance_from_center, hoi[0].lane_pose.relative_heading)
    #lane_pose = lanesegment.lane_pose_from_SE2Transform(pose_object)    
    #center_points.append(lane_pose.center_point)
    #print(relTimestamps[counter])

    counter += 1
    print(' ')
    print(counter)


('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.47744881651735316,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.06716613264383445,relative_heading=-2.892312645797762,distance_from_left=0.28944881651735316,distance_from_right=0.6654488165173531,distance_from_center=0.47744881651735316,center_point=SE2Transform([-0.43337730763613586, -0.22716486969019545],-0.23864977038437002),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=True,lateral_inside=True,outside_left=False,outside_right=False,lateral=-0.04219041719699211,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.9495352488202597,relative_heading=0.2510293000334121,distance_from_left=0.23019041719699213,distance_from_right=0.14580958280300788,distance_from_center=0.04219041719699211,center

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.45847777760966857,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.03453851853458748,relative_heading=-3.0017620289919766,distance_from_left=0.27047777760966857,distance_from_right=0.6464777776096686,distance_from_center=0.45847777760966857,center_point=SE2Transform([-0.46552486382688035, -0.2216964128013356],-0.12271972783373057),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=True,lateral_inside=True,outside_left=False,outside_right=False,lateral=-0.02088000186303199,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=1.0327282500421113,relative_heading=0.14094794320420748,distance_from_left=0.208880001863032,distance_from_right=0.167119998136968,distance_from_center=0.02088000186303199,center_po

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.45257928098995126,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.015736834793828113,relative_heading=-3.0569038308355414,distance_from_left=0.26457928098995126,distance_from_right=0.6405792809899513,distance_from_center=0.45257928098995126,center_point=SE2Transform([-0.4842671708407886, -0.22024804496191697],-0.0559149658063362),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=True,lateral_inside=True,outside_left=False,outside_right=False,lateral=-0.013662018097156864,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=1.0808032340615297,relative_heading=0.08525286866043066,distance_from_left=0.20166201809715686,distance_from_right=0.17433798190284314,distance_from_center=0.013662018097156864,cen

('tilemap', 'tile-3-0', 'straight', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.4396581168174739,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.7948717623949051,relative_heading=-3.071063922168467,distance_from_left=0.2516581168174739,distance_from_right=0.627658116817474,distance_from_center=0.4396581168174739,center_point=SE2Transform([0.2948717623949051, -0.2199999988079071],0.0),correct_direction=False)
('tilemap', 'tile-3-0', 'straight', 'lane2')
LanePose(inside=True,lateral_inside=True,outside_left=False,outside_right=False,lateral=0.0003418831825260371,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.2051282376050949,relative_heading=0.07052873142132642,distance_from_left=0.18765811681747396,distance_from_right=0.18834188318252604,distance_from_center=0.0003418831825260371,center_point=SE2Transform([-0.294

('tilemap', 'tile-3-0', 'straight', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.4345299116373058,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.7658119330406192,relative_heading=-3.0321236810002534,distance_from_left=0.2465299116373058,distance_from_right=0.6225299116373058,distance_from_center=0.4345299116373058,center_point=SE2Transform([0.26581193304061923, -0.2199999988079071],0.0),correct_direction=False)
('tilemap', 'tile-3-0', 'straight', 'lane2')
LanePose(inside=True,lateral_inside=True,outside_left=False,outside_right=False,lateral=0.005470088362694192,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.23418806695938077,relative_heading=0.10946897258954011,distance_from_left=0.1825299116373058,distance_from_right=0.1934700883626942,distance_from_center=0.005470088362694192,center_point=SE2Transform([-0.265

...

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.23428738812111588,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.3800623611924569,relative_heading=-2.926044489963114,distance_from_left=0.04628738812111588,distance_from_right=0.4222873881211159,distance_from_center=0.23428738812111588,center_point=SE2Transform([-0.22605660206834405, -0.43839632287771674],-1.3504096512944734),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.20133002824039634,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.15905170300535168,relative_heading=0.21308692033856705,distance_from_left=0.013330028240396335,distance_from_right=0.3893300282403963,distance_from_center=0.20133002824039634,center

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.23428738812111588,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.3800623611924569,relative_heading=-2.9246175549303923,distance_from_left=0.04628738812111588,distance_from_right=0.4222873881211159,distance_from_center=0.23428738812111588,center_point=SE2Transform([-0.22605660206834405, -0.43839632287771674],-1.3504096512944734),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.20133002824039634,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.15905170300535168,relative_heading=0.214513855371289,distance_from_left=0.013330028240396335,distance_from_right=0.3893300282403963,distance_from_center=0.20133002824039634,center_

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.23428738812111588,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.3800623611924569,relative_heading=-2.9234141403755833,distance_from_left=0.04628738812111588,distance_from_right=0.4222873881211159,distance_from_center=0.23428738812111588,center_point=SE2Transform([-0.22605660206834405, -0.43839632287771674],-1.3504096512944734),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.20133002824039634,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.15905170300535168,relative_heading=0.2157172699260978,distance_from_left=0.013330028240396335,distance_from_right=0.3893300282403963,distance_from_center=0.20133002824039634,center

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.23428738812111588,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.3800623611924569,relative_heading=-2.92254178620545,distance_from_left=0.04628738812111588,distance_from_right=0.4222873881211159,distance_from_center=0.23428738812111588,center_point=SE2Transform([-0.22605660206834405, -0.43839632287771674],-1.3504096512944734),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.20133002824039634,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.15905170300535168,relative_heading=0.21658962409623128,distance_from_left=0.013330028240396335,distance_from_right=0.3893300282403963,distance_from_center=0.20133002824039634,center_

('tilemap', 'tile-2-0', 'curve_right', 'lane1')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.23428738812111588,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.3800623611924569,relative_heading=-2.9220361521842166,distance_from_left=0.04628738812111588,distance_from_right=0.4222873881211159,distance_from_center=0.23428738812111588,center_point=SE2Transform([-0.22605660206834405, -0.43839632287771674],-1.3504096512944734),correct_direction=False)
('tilemap', 'tile-2-0', 'curve_right', 'lane2')
LanePose(inside=False,lateral_inside=False,outside_left=True,outside_right=False,lateral=0.20133002824039634,lateral_left=0.188,lateral_right=-0.188,along_inside=True,along_before=False,along_after=False,along_lane=0.15905170300535168,relative_heading=0.2170952581174646,distance_from_left=0.013330028240396335,distance_from_right=0.3893300282403963,distance_from_center=0.20133002824039634,center

In [7]:
#sequence = dw.SampledSequence(seq_me.timestamps, seq_me.center_points)
#m.set_object("projection2", dw.PlacedObject(), ground_truth=seq_me)
m = dw.load_map('ethz_amod_lab_k31')

m.set_object("emded", Person(0.1), ground_truth=seq_me)
ipython_draw_html(m);


INFO:dt-world:loading map ethz_amod_lab_k31
INFO:dt-world:area: RectangularArea(pmin=[0. 0.],pmax=[3.45617996 3.45617996])
INFO:dt-world:Written SVG to out/ipython_draw_html/140261094075664/drawing.svg
INFO:dt-world:Written HTML to out/ipython_draw_html/140261094075664/drawing.html
